In [12]:
import pandas as pd
import ast

# Cargar CSV
df = pd.read_csv("MMB_21K_2025_raw.csv")

# Asegurar que 'calculatedTimes' no tenga nulos
df = df[df['calculatedTimes'].notna()]

# Convertir 'calculatedTimes' a diccionario
df['calculatedTimes'] = df['calculatedTimes'].apply(ast.literal_eval)

# Extraer tiempo chip desde el campo Meta.net (milisegundos)
df['tiempo_chip_ms'] = df['calculatedTimes'].apply(lambda x: x.get('Meta', {}).get('net'))

# Convertir a segundos
df['tiempo_chip_seg'] = df['tiempo_chip_ms'] / 1000

# Convertir a formato hh:mm:ss (si lo quieres conservar)
df['tiempo_chip_str'] = pd.to_timedelta(df['tiempo_chip_ms'], unit='ms')

# Convertir 'custom-rankings' a lista de diccionarios
df['custom-rankings'] = df['custom-rankings'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[{") else [])

# Extraer la categoría desde custom-rankings
def extraer_categoria_custom(rankings):
    for item in rankings:
        if isinstance(item, dict) and item.get("type") == "category":
            return item.get("type-value")
    return None

df['categoria'] = df['custom-rankings'].apply(extraer_categoria_custom)

# Filtrar y renombrar columnas
df_final = df[[
    'fullName',
    'gender',
    'dorsal',
    'tiempo_chip_seg',
    'tiempo_chip_ms',
    'categoria',
    'gunTime'
]].rename(columns={
    'fullName': 'Nombre',
    'gender': 'Genero',
    'dorsal': 'BIB',
    'tiempo_chip_seg': 'Tiempo Chip',
    'tiempo_chip_ms': 'Paso medio',
    'categoria': 'Categoria',
    'gunTime': 'Tiempo Oficial'
})

# Exportar a nuevo CSV
df_final.to_csv("mmb2024_procesado.csv", index=False)


C:\Users\lccas\AppData\Local\Temp\ipykernel_1720\3117209747.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("MMB_21K_2025_raw.csv")
